In [1]:
import torch
from newlm.lm.elmo.lm_builder import ELMOLMBuilder
from transformers import BertLMHeadModel

In [2]:
model_path = "./outputs/en.100-percent.bert-causal.1M"
config_path = 'examples/configs_gcloud/run-100-percent.bert-causal.yaml'
model_type = "bert-causal"

### Model Bert Causal

In [3]:
model = BertLMHeadModel.from_pretrained(
    model_path
) # use pre-trained model

In [4]:
model.eval()
print("model in eval mode for consistency")

model in eval mode for consistency


In [5]:
# model.__dict__

#### Helpers

In [6]:
from newlm.utils.file_util import read_from_yaml
config_file = read_from_yaml(config_path)

In [7]:
# trainer (helper)
from transformers import TrainingArguments, Trainer
trainer_args = TrainingArguments(output_dir="tmpout",**config_file['lm']['hf_trainer']['args'])

# lm builder (helper)
elmo_lm_builder = ELMOLMBuilder(
    model_config = config_file['lm']['model']['config'],
    tokenizer=model_path, # use pre-trained tokenizer
    model_type=model_type,
    max_len=128
)

In [8]:
%%capture
from newlm.glue.cls_trainer import ClsTrainer
cls_trainer = ClsTrainer(
    model_path,
    model_path,
    model_type=model_type
)

## Data

In [9]:
%%capture
ds_ori = cls_trainer.helper("mrpc", {})

In [10]:
ds_ori['train']['sentence1'][:3]

['Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 "Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .",
 'They had published an advertisement on the Internet on June 10 , offering the cargo for sale , he added .']

In [11]:
%%capture
ds_moses = cls_trainer.helper("mrpc", {"detokenizer": "moses"})

2021-12-01 17:16:05.015 | INFO     | newlm.glue.cls_trainer:helper:71 - Use detokenizer moses
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [12]:
ds_moses['train']['sentence1'][:3]

['Amrozi accused his brother, whom he called "the witness", of deliberately distorting his evidence.',
 "Yucaipa owned Dominick's before selling the chain to Safeway in 1998 for $2.5 billion.",
 'They had published an advertisement on the Internet on June 10, offering the cargo for sale, he added.']

In [13]:
%%capture
ds_tb = cls_trainer.helper("mrpc", {"detokenizer": "treebank"})

2021-12-01 17:16:08.143 | INFO     | newlm.glue.cls_trainer:helper:71 - Use detokenizer treebank


In [14]:
ds_tb['train']['sentence1'][:3]

['Amrozi accused his brother, whom he called " the witness ", of deliberately distorting his evidence.',
 "Yucaipa owned Dominick's before selling the chain to Safeway in 1998 for $2.5 billion.",
 'They had published an advertisement on the Internet on June 10, offering the cargo for sale, he added.']

In [15]:
trainer = Trainer(model=model, args=trainer_args, data_collator=elmo_lm_builder.data_collator, train_dataset=ds_ori['train'])
dl_ori = trainer.get_train_dataloader() 

max_steps is given, it will override any value given in num_train_epochs
The following columns in the training set  don't have a corresponding argument in `BertLMHeadModel.forward` and have been ignored: idx, sentence1, sentence2.


In [16]:
trainer = Trainer(model=model, args=trainer_args, data_collator=elmo_lm_builder.data_collator, train_dataset=ds_moses['train'])
dl_moses = trainer.get_train_dataloader() 

max_steps is given, it will override any value given in num_train_epochs
The following columns in the training set  don't have a corresponding argument in `BertLMHeadModel.forward` and have been ignored: idx, sentence1, sentence2.


In [17]:
trainer = Trainer(model=model, args=trainer_args, data_collator=elmo_lm_builder.data_collator, train_dataset=ds_tb['train'])
dl_tb = trainer.get_train_dataloader() 

max_steps is given, it will override any value given in num_train_epochs
The following columns in the training set  don't have a corresponding argument in `BertLMHeadModel.forward` and have been ignored: idx, sentence1, sentence2.


In [18]:
batch_ori = next(iter(dl_ori))
batch_moses = next(iter(dl_moses))
batch_tb = next(iter(dl_tb))

In [19]:
model.eval()
print("model in eval mode for consistency")

model in eval mode for consistency


### RUN

In [20]:
del batch_ori['label']
batch_ori['input_ids']

tensor([[    2,  1785,  3719,  ...,  2100,    18,     3],
        [    2,  1771,  2957,  ...,     0,     0,     0],
        [    2,  1771, 19714,  ...,     0,     0,     0],
        ...,
        [    2,  9344,    11,  ...,     0,     0,     0],
        [    2,  1833,  6441,  ...,     0,     0,     0],
        [    2,  2195,  4241,  ...,     0,     0,     0]])

In [21]:
res_ori = model(**batch_ori)

In [26]:
res_ori.loss

tensor(4.5310, grad_fn=<NllLossBackward>)

In [22]:
del batch_moses['label']
batch_moses['input_ids']

tensor([[    2,  1785,  3719,  ...,  2100,    18,     3],
        [    2,  1771,  2957,  ...,     0,     0,     0],
        [    2,  1771, 19714,  ...,     0,     0,     0],
        ...,
        [    2,  9344,    11,  ...,     0,     0,     0],
        [    2,  1833,  6441,  ...,     0,     0,     0],
        [    2,  2195,  4241,  ...,     0,     0,     0]])

In [23]:
res_moses = model(**batch_moses)

In [27]:
res_moses.loss

tensor(4.5310, grad_fn=<NllLossBackward>)

In [24]:
del batch_tb['label']
batch_tb['input_ids']

tensor([[    2,  1785,  3719,  ...,  2100,    18,     3],
        [    2,  1771,  2957,  ...,     0,     0,     0],
        [    2,  1771, 19714,  ...,     0,     0,     0],
        ...,
        [    2,  9344,    11,  ...,     0,     0,     0],
        [    2,  1833,  6441,  ...,     0,     0,     0],
        [    2,  2195,  4241,  ...,     0,     0,     0]])

In [25]:
res_tb = model(**batch_tb)

In [28]:
res_tb.loss

tensor(4.5310, grad_fn=<NllLossBackward>)